<a href="https://colab.research.google.com/github/Revice-lee/project_Youtube/blob/main/KEYBERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C_test02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

In [ ]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()
bot_con = pd.read_csv('./youtube_bot_data_refined.csv')
bot_con

Saving youtube_bot_data_refined.csv to youtube_bot_data_refined.csv


,timestamp,_id,_index,_score,_type,break_first_ad_name,break_first_ad.keyword,channel,describe,first_ad,first_ad_name,keyword,likes,persona,session,title,upload,url,viewership
0,"Dec 11, 2023 @ 10:50:22.579",h02RVowBHud6z_yDdmJb,youtube_bot_data,-,_doc,-,-,9 malam,-,-,-,의상아이디어,"19,850",아바 스타일 새비,501,"스트릿패션 * 의상 아이디어 * 옷 매칭+쉬운 스트리트 스타일 의상, 십대 옷 코디...","Jul 19, 2021 @ 09:00:00.000",https://www.youtube.com/watch?v=Ps-0f0K6izM,296
1,"Dec 11, 2023 @ 10:50:22.579",iE2RVowBHud6z_yDdmJp,youtube_bot_data,-,_doc,-,-,Club Crochet,🌎 Donate to Download: http://www.clubcrochet.c...,-,-,의상아이디어,469,아바 스타일 새비,360,LIVE CROCHET ALONG - Tuck the Giant Tortoise -...,"May 4, 2023 @ 09:00:00.000",https://www.youtube.com/watch?v=QW5JZeLBKGo,"93,443"
2,"Dec 11, 2023 @ 10:50:22.579",iU2RVowBHud6z_yDdmJ0,youtube_bot_data,-,_doc,-,-,Club Crochet,Get the pattern here: http://www.clubcrochet.c...,-,-,의상아이디어,310,아바 스타일 새비,180,Amigurumi Space Themed Baby Mobile - Space Liv...,"Apr 11, 2021 @ 09:00:00.000",https://www.youtube.com/watch?v=XnU72WVPUcQ,"13,710"
3,"Dec 11, 2023 @ 10:50:22.579",ik2RVowBHud6z_yDdmJ_,youtube_bot_data,-,_doc,-,-,Hanabira工房,今年の8月、たまたまリサイクルショップを訪れたところ隅の方に埃をかぶって置かれているワープロ...,-,-,의상아이디어,"23,949",아바 스타일 새비,480,DIY☺︎隅に置かれた壊れたワープロにミニチュアを起動させたい。second-hand goods,"Nov 17, 2023 @ 09:00:00.000",https://www.youtube.com/watch?v=-Y2FVB0nj_Q,"485,303"
4,"Dec 11, 2023 @ 10:50:22.579",i02RVowBHud6z_yDdmKL,youtube_bot_data,-,_doc,-,-,Club Crochet,👍 Get this to 350 likes for another giveaway n...,-,-,의상아이디어,561,아바 스타일 새비,420,Crocheting Amigurumi Ghosts - Live Crochet Along,"Oct 10, 2021 @ 09:00:00.000",https://www.youtube.com/watch?v=2F8LdOx5RJA,"7,694"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7247,"Dec 9, 2023 @ 21:48:42.546",FE2fTowBHud6z_yDdz9w,youtube_bot_data,-,_doc,-,-,빨강도깨비,좀비 영화에는 빠지지 않고 등장하는 무기가 하나 있습니다. 바로 샷건이죠. 실제...,-,-,서바이벌,"11,721",액션앤드류,120,좀비영화에서 이 총이 빠지지 않는 의외의 이유,"Feb 25, 2022 @ 09:00:00.000",https://www.youtube.com/watch?v=sjlyrX12VGU,"984,046"
7248,"Dec 9, 2023 @ 21:48:42.546",FU2fTowBHud6z_yDdz-A,youtube_bot_data,-,_doc,-,-,빨강도깨비,전 세계 40여 개국에서 사용되는 가장 보편화 된 유탄 발사기이자 누구나 사용할 수...,-,-,서바이벌,"14,960",액션앤드류,240,헐리우드판 RPG와 현실의 차이,"May 14, 2021 @ 09:00:00.000",https://www.youtube.com/watch?v=-SotR1tdMog,"1,494,391"
7249,"Dec 9, 2023 @ 21:48:42.546",Fk2fTowBHud6z_yDdz-L,youtube_bot_data,-,_doc,-,-,더 벙커 : 전쟁사관학교,"소총의 역사 2편, AR계열과 한국 총기 개발사",-,-,서바이벌,"4,740",액션앤드류,180,"소총의 역사 2편, AR계열과 한국 총기 개발사","Sep 17, 2023 @ 09:00:00.000",https://www.youtube.com/watch?v=uuwBbHia64U,"398,381"
7250,"Dec 9, 2023 @ 21:44:05.342",Ak2bTowBHud6z_yDPD8w,youtube_bot_data,-,_doc,-,-,14F 일사에프,‘WA BAR’부터 ‘인디카 IPA’까지... 한국인은 맥주를 어떻게 소비했을까요?...,-,-,수제 맥주,"2,340",브루마스터 벤,240,당신을 위로해 줄 한잔! 국내 수제 맥주 이야기 떠먹여 드립니다🥄ㅣ돈슐랭 / 14F,"Feb 22, 2021 @ 09:00:00.000",https://www.youtube.com/watch?v=RmeQbFEhrlw,"210,213"


In [ ]:
!pip install emoji

In [ ]:
from emoji import core
import re
#bot_con['describe_emoji_repalced'] = core.replace_emoji(bot_con['describe'], replace = "")
#print(bot_con['describe_emoji_repalced'])
bot_con['describe_emoji_replaced'] = bot_con['describe']

def describe_emoji_repalce():

  s = bot_con['describe_emoji_replaced'].count()

  for i in range(s):
    des_text = bot_con['describe_emoji_replaced'][i]
    des_text = core.replace_emoji(des_text, replace = " ")
    des_text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", des_text)
    des_text = re.sub("\n", "", des_text)
    bot_con['describe_emoji_replaced'][i] = des_text

  return bot_con['describe_emoji_replaced']

describe_emoji_repalce()

bot_con['title_emoji_replaced'] = bot_con['title']

def title_emoji_repalce():

  s = bot_con['title_emoji_replaced'].count()

  for i in range(s):
    tit_text = bot_con['title_emoji_replaced'][i]
    tit_text = core.replace_emoji(tit_text, replace = " ")
    tit_text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", tit_text)
    tit_text = re.sub("\n", "", tit_text)
    bot_con['title_emoji_replaced'][i] = tit_text

  return bot_con['title_emoji_replaced']

title_emoji_repalce()

0       스트릿패션  의상 아이디어  옷 매칭쉬운 스트리트 스타일 의상 십대 옷 코디  Ou...
1       LIVE CROCHET ALONG  Tuck the Giant Tortoise  L...
2       Amigurumi Space Themed Baby Mobile  Space Live...
3                                    DIY secondhand goods
4         Crocheting Amigurumi Ghosts  Live Crochet Along
                              ...                        
7247                            좀비영화에서 이 총이 빠지지 않는 의외의 이유
7248                                    헐리우드판 RPG와 현실의 차이
7249                            소총의 역사 2편 AR계열과 한국 총기 개발사
7250          당신을 위로해 줄 한잔 국내 수제 맥주 이야기 떠먹여 드립니다 돈슐랭  14F
7251                           맥주에 이걸 넣었더니 생긴 변화 돈슐랭  14F
Name: title_emoji_replaced, Length: 7252, dtype: object

In [ ]:
bot_con['describe_only_korean'] = bot_con['describe_emoji_replaced'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
bot_con['title_only_korean'] = bot_con['title_emoji_replaced'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [ ]:
#doc = bot_con['describe'][1]
doc = bot_con['describe'][3]

In [ ]:
n_gram_range = (3, 3)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 116
trigram 다섯개만 출력 : ['00 壊れたワープロ紹介 18' '00 水槽作り 36' '01 カウンターテーブル ミニチュア作り' '03 キャビネット作り 22'
 '06 パソコンデスク ミニチュア作り']


In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [ ]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['お知り合いのプロにお任せしました リヴリーアイランド のお客様にはhanabira工房の動画を見てくれる方が多いらしくとても嬉しく思います', '古くて壊れたワープロの分解は素人がやると非常に危険なので お知り合いのプロにお任せしました リヴリーアイランド', '2ヶ月以上掛かりましたが壊れたワープロは無事起動しました 古くて壊れたワープロの分解は素人がやると非常に危険なので お知り合いのプロにお任せしました', 'たまたまリサイクルショップを訪れたところ隅の方に埃をかぶって置かれているワープロを発見 2ヶ月以上掛かりましたが壊れたワープロは無事起動しました 古くて壊れたワープロの分解は素人がやると非常に危険なので', '今年の8月 たまたまリサイクルショップを訪れたところ隅の方に埃をかぶって置かれているワープロを発見 2ヶ月以上掛かりましたが壊れたワープロは無事起動しました']


In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings,
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['錬金術からうまれた不思議な生き物 リヴリー のんびり気ままな生活を送りませんか',
 'チャンネル登録よろしくお願いします また リクエストなどがありましたらどしどしコメント欄に書いていってください',
 'また リクエストなどがありましたらどしどしコメント欄に書いていってください 答えられる範囲内で作って動画にあげたいと思います',
 '古くて壊れたワープロの分解は素人がやると非常に危険なので お知り合いのプロにお任せしました リヴリーアイランド',
 '今年の8月 たまたまリサイクルショップを訪れたところ隅の方に埃をかぶって置かれているワープロを発見 2ヶ月以上掛かりましたが壊れたワープロは無事起動しました']

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20)

['hanabira_kobo チャンネル登録よろしくお願いします また',
 '提供 リヴリーアイランド ダウンロードはこちら',
 '壊れたワープロ紹介 18 キーボード掃除',
 'のお客様にはhanabira工房の動画を見てくれる方が多いらしくとても嬉しく思います ありがとうございます 提供',
 '今年の8月 たまたまリサイクルショップを訪れたところ隅の方に埃をかぶって置かれているワープロを発見 2ヶ月以上掛かりましたが壊れたワープロは無事起動しました']

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['今年の8月 たまたまリサイクルショップを訪れたところ隅の方に埃をかぶって置かれているワープロを発見 2ヶ月以上掛かりましたが壊れたワープロは無事起動しました',
 'たまたまリサイクルショップを訪れたところ隅の方に埃をかぶって置かれているワープロを発見 2ヶ月以上掛かりましたが壊れたワープロは無事起動しました 古くて壊れたワープロの分解は素人がやると非常に危険なので',
 '2ヶ月以上掛かりましたが壊れたワープロは無事起動しました 古くて壊れたワープロの分解は素人がやると非常に危険なので お知り合いのプロにお任せしました',
 '古くて壊れたワープロの分解は素人がやると非常に危険なので お知り合いのプロにお任せしました リヴリーアイランド',
 'お知り合いのプロにお任せしました リヴリーアイランド のお客様にはhanabira工房の動画を見てくれる方が多いらしくとても嬉しく思います']

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['今年の8月 たまたまリサイクルショップを訪れたところ隅の方に埃をかぶって置かれているワープロを発見 2ヶ月以上掛かりましたが壊れたワープロは無事起動しました',
 'com event sanrio',
 'twitter hanabira7878 instagram',
 'app dbg 2024',
 'いろいろ設置 41 41']